

Estudando os melhores atributos e hiperparâmetros do regressor de floresta aleatória
===================

### Autores: Glauber Nascimento, Rafael Anis, Maria Emily Gomes

<p style="text-align:justify;"> Nesse notebook, utilizou-se o banco de dados $MSK MetTropism$, o qual foi produzido e estudado pelo grupo o qual publicou "Genomic characterization of metastatic patterns from prospective clinical sequencing of 25,000 patients". Nosso projeto, no entanto, irá utilizar essa vasta informação e tentar predizer o tempo de sobrevida de pacientes de <strong>Câncer de pulmão de células não pequenas</strong>. E, nesse notebook, a itenção é contar como que alcançamos nossos resultados e o porquê de ter escolhido o regressor de floresta aleatória. Esse modelo pretende discutir o tempo de sobrevida de pacientes que não foram previstos como curados, ou seja, apresentaram um tempo de sobrevida maior que 60 meses no nosso outro modelo <strong>modelo []</strong>.

<p style="text-align:justify;"> Para conseguir os melhores resultados, não só especificamos um tipo de câncer para treinar o modelo, como também escolhemos dois focos de estudos: <strong>modelo de floresta aleatória</strong> e o <strong>modelo KNN</strong> e dividimos as observações do dataset em dados de treino e de teste para verificar a performance. Além disso, houve um estudo acerca dos melhores atributos - assim como verificamos se havia multicolinearidade associada ao banco de dados, dos melhores parâmetros com o optuna e, assim, espera-se um RMSE baixo, a modo que os resultados se mostrem relevantes. Para haver um métrica que nos possibilite obter uma sensibilidade para a qualidade do modelo, iremos obter o RMSE de um modelo baseline. Como a métrica do modelo de floresta aleatória, a qual avalia o desvio padrão quadrado, foi menor que a do baseline, o trabalho apresentara discurssões de como alcançou o resultado e como poderia aprimorá-lo.

### Importando as bibliotecas 

<p style="text-align:justify;">  A biblioteca <strong>pandas</strong> será necessária para ler e editar o dataframe utilizado neste trabalho. 

In [1]:
import pandas as pd
import numpy as np

### Leitura do dataset

<p style="text-align:justify;"> O dataset a seguir é de onde foi realizada todas as operações necessárias para desenvolver os modelos. O primeiro foco do grupo foi encontrar os atributos relevantes para o objetivo, como também encontrar uma coluna de target adequada para a nossa discussão.

In [2]:
df = pd.read_csv("msk_met_2021_clinical_data (1).tsv", sep = "\t")

<p style="text-align:justify;"> Em seguida, discutiu-se acerca da coluna de target, pois a sobrevida encontrada no dataset não era coerente ao dado que precisávamos para a predição. Então, a seguir, será observado como resolvemos o empasse, assim como o estudo do <strong>[notebook bolinha]</strong> mostra os tais problemas e o raciocínio detalhado para resolvé-lo. Em resumo, teremos uma coluna de sobrevida que avalia o tempo entre o <strong>sequenciamento genômico do paciente até o mês que foi identificado a sua morte.</strong> Contudo, produzir um modelo que prediz um tempo de vida maior que 60 meses não é pertinente, pois o tempo que observa-se um paciente de câncer é até 60 meses. Nesse sentido informar uma sobrevida de 60 ou, por exemplo, 80 meses implica na mesma situação: o paciente estará curado com os atributos que possui. 

In [3]:
df['Curado'] = pd.NA
df ['Sobrevivencia (Meses)'] = pd.NA

for i, meses in enumerate(df['Overall Survival (Months)']):
    
    if meses >= 60:
        df.loc [i, 'Curado'] = 1
        
    else:
        if df.loc [i, 'Overall Survival Status'] == '1:DECEASED':
            df.loc [i, 'Curado'] = 0
            df.loc [i, 'Sobrevivencia (Meses)'] = meses

<p style="text-align:justify;"> Para minimizar o RMSE, optamos por selecionar um câncer específico do dataset. Considerando que o Câncer de pulmão de células não pequenas é o que apresenta a maior quantidade de observações, o predição da sobrevida de pacientes com esse tipo de câncer foi o selecionada para ser o objeto de estudo. Assim, o foco novamente foi a busca por um modelo de baixo viés. 

In [4]:
df = df.loc[df["Cancer Type"] == "Non-Small Cell Lung Cancer"]

#### Separando as colunas pertinentes para o modelo em atributos e target

<p style="text-align:justify;"> Usaremos para o modelo atributos como o <strong>Fraction Genome Altered</strong>, o qual é a razão entre a soma dos segmentos do genoma que foram alterados pele tamanho total do genoma do paciente, esse resultado é bastante relevante para identificar o quão instável está o tumor e, desse modo, saber como está a sua progressão. Há também atributos que conseguem identificar a situação metastática do câncer a partir das informações genômicas, pois há como identificar uma hipermutação, MSI-High (MSI-H), a partir do <strong>MSI Score</strong>, Microsatellite instability, a instabilidade de microssatélites, ou para a quantidade de mutações, informação essa que pode ser encontrada na coluna <strong>Mutation Count</strong>. O <strong>TMB (nonsynonymous)</strong>, tumor mutational burden, quantifica o número total de mutações encontradas no DNA do tumor e é considerado alto para valores maior que 10 mut/Mb, o que foi bastante encontrado em pacientes com adenocarcinoma de pulmão. A <strong>Age at Surgical Procedure</strong>, idade da operação cirúrgica, é relavante para qualquer paciente, pois, com idades distintas, os pacientes apresentam resposta imunológicas diferentes. E, considerando a relevante relação entre quantidade de mutações e a sobrevida, iremos utilizar também os atributos <strong>Met Count</strong> e <strong>Met Site Count</strong>, que, respectivamente, a quantidade de mutações e a quantidade de sítios de mutações.

In [5]:
ATRIBUTOS = [                  
        "Age at Surgical Procedure",           
        "Fraction Genome Altered",              
        "Met Count",                            
        "Met Site Count",                       
        "MSI Score",                   
        "Mutation Count",              
        "TMB (nonsynonymous)"]         
TARGET = ["Sobrevivencia (Meses)"]

#### Encontrando o dataset com os dados que serão utilizados

<p style="text-align:justify;"> O dataset a seguir é o tratado, ou seja, haverá apenas pacientes que tiveram um sobrevida menor que 60 meses, com colunas selecionadas nos debates anteriores, com a eliminação de colunas que não apresentavam valores e a conversão dos valores. 

In [6]:
df = df.loc[df["Sobrevida"] == 0]

In [7]:
df = df.reindex(ATRIBUTOS + TARGET, axis=1)
df = df.dropna()
df = df.convert_dtypes()

In [8]:
df

,Age at Surgical Procedure,Fraction Genome Altered,Met Count,Met Site Count,MSI Score,Mutation Count,TMB (nonsynonymous),Sobrevivencia (Meses)
44,71.17,0.2474,4,4,0.44,15,16.63733,47.9
60,79.31,0.0026,7,7,0.0,4,4.436621,19.58
69,50.73,0.4502,8,8,3.36,5,4.436621,18.73
79,37.69,0.3584,4,4,0.93,3,3.327466,0.0
105,69.42,0.4041,7,7,1.33,3,2.936159,13.21
...,...,...,...,...,...,...,...,...
25485,63.46,0.2645,4,4,0.18,10,8.646981,1.31
25508,67.29,0.5715,1,1,0.43,19,16.429264,0.66
25535,55.57,0.1114,11,8,0.29,4,3.458792,8.51
25536,61.42,0.1031,3,3,0.05,1,0.864698,10.87


<p style="text-align:justify;"> Estudando o artigo de referência que disponibilou o dataset utilizado para encontrar o modelo que melhor descreve a sobrevida de um paciente de câncer de pulmão de células não pequenas,$\textit{Non-Small Cell Lung}$, separou-se as colunas de atributos mostradas no tópico anterior. Contudo, para evitar um custo computacional desnecessário, será selecionado atributos que não apresentem multicolinearidade, ou seja, não é possível com uma combinação linear de atributos prever um outro. Isso será realizado com a seleção de atributos pelo fator de inflação de variância (VIF), desse modo, evita-se dados redundantes e cada operação realizada pelo modelo será relevante para encontrar o targer proposto.  

#### Separando dados de treino e teste

<p style="text-align:justify;"> Separa-se dados de treino e de teste para avaliar a performance do trabalho desenvolvido. Mesmo com o tratamento de dados utilizados, ainda há muitos dados para serem utilizados, desse modo, mostra-se suficiente um teste que represente 10% das observações que o dataset base para o modelo possui.

In [9]:
from sklearn.model_selection import train_test_split

TAMANHO_TESTE = 0.1
SEMENTE_ALEATORIA = 61455

indices = df.index
indices_treino, indices_teste = train_test_split(
    indices, test_size=TAMANHO_TESTE, random_state=SEMENTE_ALEATORIA
)

df_treino = df.loc[indices_treino]
df_teste = df.loc[indices_teste]

X_treino = df_treino.reindex(ATRIBUTOS, axis=1).values
y_treino = df_treino.reindex(TARGET, axis=1).values.ravel()

X_teste = df_teste.reindex(ATRIBUTOS, axis=1).values
y_teste = df_teste.reindex(TARGET, axis=1).values.ravel()

#### Seleção de atributos pelo fator de inflação de variância (VIF)

O **fator de inflação de variância** (*variance inflation factor*) é um algoritmo que encontra a multicolinearidade de um conjunto de dados. Os passos a seguir desse código foram descritos pelo professor Daniel Cassar:

1.  Escolha o valor do limiar do $\mathrm{VIF}$.

<p>Escolheu-se o mais rigoroso, pois esses atributos já foram selecionados de acordo com o bom senso para modelar o fenômeno. Além disso, após essa etapa, há as operações algébricas que o código realiza, não há mais escolha pensada no problema.</p>
   
2.  Para cada atributo disponível, considere ele como target e treine um modelo linear com os atributos restantes. Registre o valor $\mathrm{VIF}$ deste modelo sendo que $\mathrm{VIF} = 1 / (1 - R^2)$

3.  Se pelo menos um dos valores $\mathrm{VIF}$ observados no passo 2 for maior que o limiar definido no passo 1, vá ao passo seguinte. Do contrário, encerre o processo

5.  Remova o atributo de maior valor $\mathrm{VIF}$ e retorne ao passo 2

In [10]:
from sklearn.ensemble import RandomForestRegressor

def selecao_vif(df_atributos, limiar_vif):
    """Realiza a seleção de atributos por VIF.

    Args:
      df_atributos: DataFrame contendo os atributos.
      limiar_vf: valor do limiar do vif. Número positivo. Usualmente é 5 ou 10.

    Returns:
      DataFrame com os atributos selecionados.

    """
    df = df_atributos.copy()

    while True:
        VIFs = []

        for col in df.columns:
            X = df.drop(col, axis=1).values
            y = df[col].values

            r_quadrado = RandomForestRegressor().fit(X, y).score(X, y) ##
            if r_quadrado != 1:
                VIF = 1 / (1 - r_quadrado)
            else:
                VIF = float("inf")

            VIFs.append(VIF)

        VIF_maximo = max(VIFs)

        if VIF_maximo > limiar_vif:
            indice = VIFs.index(VIF_maximo)
            coluna_remocao = df.columns[indice]
            df = df.drop(coluna_remocao, axis=1)

        else:
            break

    return df

In [11]:
LIMIAR_VIF = 10

X_treino = pd.DataFrame(X_treino)
X_teste = pd.DataFrame(X_teste)

X_treino_modificado = selecao_vif(X_treino, LIMIAR_VIF)
X_teste_modificado = X_teste.reindex(X_treino_modificado.columns, axis=1)

In [12]:
X_treino_modificado

,0,2,4,6
0,87.42,2,0.06,7.782283
1,73.29,3,0.0,3.914879
2,76.24,1,0.0,5.188189
3,80.6,2,0.0,5.872318
4,60.29,2,0.63,17.293962
...,...,...,...,...
1732,31.42,4,3.91,7.764087
1733,79.18,4,0.15,2.936159
1734,67.52,6,0.75,5.872318
1735,79.86,3,7.61,34.587924


<p style="text-align:justify;"> Escolheu-se o limiar VIF como 10 por ser padrão e conseguiu-se bons resultados com esse valor. Após a eliminação de atributos que apresentavam multicolinearidade, renova-se os atributos. Os que estão comentados são os atributos que não serão mais utilizados.

In [13]:
ATRIBUTOS_VIF = [                  
        "Age at Surgical Procedure",           
        # "Fraction Genome Altered",              
        "Met Count",                            
        # "Met Site Count",                       
        "MSI Score",                   
        #"Mutation Count",              
        "TMB (nonsynonymous)"
]         
TARGET = ["Sobrevivencia (Meses)"]

### Modelo Baseline

O modelo baseline para encontrar a sobrevida de pacientes que o primeiro modelo encontrou que não serão curadas pelo tratamento o qual estão realizando, será para poder obter um RMSE comparativo. O objetico é que o modelo de floresta aleatória, o qual investida vários cenários e consegue, assim, modelar cenários complexos com diversas variáveis, consiga apresentar um RMSE muito menor.

In [14]:
from sklearn.dummy import DummyRegressor

x = df.reindex(ATRIBUTOS_VIF, axis=1)
y = df.reindex(TARGET, axis=1)

modelo = DummyRegressor()

modelo.fit(x, y)

y_previsto = modelo.predict(x)

print(y_previsto)

[14.22521491 14.22521491 14.22521491 ... 14.22521491 14.22521491
 14.22521491]


In [15]:
from sklearn.metrics import mean_squared_error

y_verdadeiro = y

RMSE = mean_squared_error(y_verdadeiro, y_previsto, squared = False)

print(f"O RMSE foi de: {RMSE}")

O RMSE foi de: 12.420902597184595


<p style="text-align:justify;"> Agora que há uma métrica para o baseline, modelo mais simples possível para prever o target, começa-se utilizar técnicas, como ao selecionar os melhores hiperparâmetros de modo que minimize ao máximo o valor do RMSE para o 
<p>
<p style="text-align:justify;"><strong>Antes de começar o estudo de hiperparâmetros, vale notar que esse modelo não tem erro associado do modelo classificador, pois os dados que foram treinados nesse notebook não são os que ele apontou como não curados, são os pacientes do dataset original que apresentaram uma sobrevida até 60 meses.</strong>

## Otimizando hiperparâmetros com o optuna para o regressor de floresta aleatória

<p style="text-align:justify;"> Encontrando o melhor conjunto de hiperparâmetros com o optuna. Esta seleção será realizada com o novo dataset que não apresenta mais multicolinearidade em seus dados, desse modo o esperado é encontrar um bom conjunto de hiperparâmetros de modo menos custoso computacionalmente.

In [16]:
from sklearn.model_selection import train_test_split

TAMANHO_TESTE = 0.1
SEMENTE_ALEATORIA = 20404

df = df.reindex(ATRIBUTOS_VIF + TARGET, axis=1)
df = df.dropna()

indices = df.index
indices_treino, indices_teste = train_test_split(
    indices, test_size=TAMANHO_TESTE, random_state=SEMENTE_ALEATORIA
)

df_treino = df.loc[indices_treino]
df_teste = df.loc[indices_teste]

X_treino = df_treino.reindex(ATRIBUTOS_VIF, axis=1).values
y_treino = df_treino.reindex(TARGET, axis=1).values.ravel()

X_teste = df_teste.reindex(ATRIBUTOS_VIF, axis=1).values
y_teste = df_teste.reindex(TARGET, axis=1).values.ravel()

X = df.reindex(ATRIBUTOS_VIF, axis=1).values
y = df.reindex(TARGET, axis=1).values.ravel()

#### Delimitando o espaço de busca 

<p style="text-align:justify;"> Inicialmente, o espaço de busca era longo e o optuna era realizado várias vezes. Isso serviu de base para entender por onde o optuna estava encontrando os melhores conjuntos e com essa informação, houve a busca a seguir perto dos valores com os melhores resultados para cada hiperparâmetro.

In [17]:
from sklearn.ensemble import RandomForestRegressor

def cria_instancia_modelo(trial):
    """Cria uma instância do modelo.

    Args:
      trial: objeto tipo Trial do optuna.

    Returns:
      Uma instância do modelo desejado.

    """
    parametros = {
        "n_estimators": trial.suggest_int("num_arvores", 10, 500),
        "criterion": trial.suggest_categorical(
            "critério", ["squared_error", "friedman_mse", "poisson"]
        ),
        "min_samples_split": trial.suggest_int(
            "min_exemplos_split", 2, 50, log=True
        ),
        "min_samples_leaf": trial.suggest_int(
            "min_exemplos_folha", 1, 100, log=True
        ),
        "max_features": trial.suggest_float("num_max_atributos", 0, 1),
        "n_jobs": -1,
        "bootstrap": True,
        "random_state": SEMENTE_ALEATORIA
    }

    model = RandomForestRegressor(**parametros)

    return model

#### Função objetivo

<p style="text-align:justify;"> A **função objetivo**  de um problema de otimização computa métrica de interesse, nesse caso, será o RMSE e, para garantir que aleatoriedade dos dados de teste não influencie tanto no valor encontrado, esse será obtido por validação cruzada. 

In [18]:
from sklearn.model_selection import cross_val_score

def funcao_objetivo(trial, X, y, num_folds):
    """Função objetivo do optuna

    Referencia:
      https://medium.com/@walter_sperat/ using-optuna-with-sklearn-the-right-way-part-1-6b4ad0ab2451

    """
    modelo = cria_instancia_modelo(trial)

    metricas = cross_val_score(
        modelo,
        X,
        y,
        scoring="neg_root_mean_squared_error",
        cv=num_folds,
    )

    return -metricas.mean()

#### Otimizando os hiperparâmetros

<p style="text-align:justify;"> O arquivo <strong>hiperparametros1</strong> guarda o melhor conjunto de hiperparâmetros de todos os estudos realizados no notebook.

In [19]:
from optuna import create_study

NOME_DO_ESTUDO = "hiperparametros1"

objeto_de_estudo = create_study(
    direction="minimize",
    study_name=NOME_DO_ESTUDO,
    storage=f"sqlite:///{NOME_DO_ESTUDO}.db",
    load_if_exists=True,
)

[I 2024-11-16 17:14:18,109] Using an existing study with name 'hiperparametros1' instead of creating a new one.


#### Função objetivo parcial, uma função objetivo que apresenta apenas o argumento `trial`

<p style="text-align:justify;"> Para conseguir rodar o otimizador, será necessário de uma função objetivo que tenha apenas um argumento, o <strong>trial</strong>. Como a <strong>funcao_objetivo</strong> definida não cumpre com este requisito, defini-se a <strong>funcao_objetivo_parcial`</strong>.

In [20]:
NUM_FOLDS = 50

def funcao_objetivo_parcial(trial):
    return funcao_objetivo(trial, X_treino, y_treino, NUM_FOLDS)

#### Define as tentativas e roda o otimizador

<p style="text-align:justify;"> Como já foram realizados inúmeros estudos, o notebook apresenta um como número de tentativas para quem queira encontrar o nosso modelo ou, por sorte, um melhor em um curto tempo, apenas rode o documento com o arquivo "hiperparametros1".

In [21]:
NUM_TENTATIVAS = 200

objeto_de_estudo.optimize(funcao_objetivo_parcial, n_trials=NUM_TENTATIVAS)

[I 2024-11-16 17:15:41,663] Trial 932 finished with value: 11.965495543012857 and parameters: {'num_arvores': 420, 'critério': 'squared_error', 'min_exemplos_split': 27, 'min_exemplos_folha': 3, 'num_max_atributos': 0.06184822158352796}. Best is trial 653 with value: 11.887704489261667.
[W 2024-11-16 17:15:55,467] Trial 933 failed with parameters: {'num_arvores': 446, 'critério': 'squared_error', 'min_exemplos_split': 30, 'min_exemplos_folha': 1, 'num_max_atributos': 0.14728730396373269} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "c:\venv\ilumpy\Lib\site-packages\optuna\study\_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "C:\Users\maria24019\AppData\Local\Temp\ipykernel_37604\1606261053.py", line 4, in funcao_objetivo_parcial
    return funcao_objetivo(trial, X_treino, y_treino, NUM_FOLDS)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Use

KeyboardInterrupt: 

In [22]:
melhor_trial = objeto_de_estudo.best_trial

print(f"Número do melhor trial: {melhor_trial.number}")
print(f"Parâmetros do melhor trial: {melhor_trial.params}")

Número do melhor trial: 653
Parâmetros do melhor trial: {'num_arvores': 448, 'critério': 'squared_error', 'min_exemplos_split': 29, 'min_exemplos_folha': 3, 'num_max_atributos': 0.12798723010757682}


### Estimando a performance do modelo floresta aleatória

<p style="text-align:justify;"> Nessa etapa, com os atributos que evitam multicolinearidade selecionados e os melhores hiperparâmentros encontrados pelo optuna, é momento de gerar o melhor modelo encontrado por esse conjunto de algorítmos. Para obter um bom estimador de performance do modelo, irá usar validação cruzada $k$​-fold.

In [23]:
from sklearn.metrics import mean_squared_error

modelo = cria_instancia_modelo(melhor_trial)
modelo.fit(X_treino, y_treino)

y_verdadeiro = y_teste
y_previsao = modelo.predict(X_teste)

RMSE = mean_squared_error(y_verdadeiro, y_previsao) ** (1/2)

print(RMSE)

13.211070024807215


In [24]:
from optuna import load_study

objeto_de_estudo_carregado = load_study(
    study_name=NOME_DO_ESTUDO,
    storage=f"sqlite:///{NOME_DO_ESTUDO}.db",
)

df = objeto_de_estudo_carregado.trials_dataframe()

df

,number,value,datetime_start,datetime_complete,duration,params_critério,params_min_exemplos_folha,params_min_exemplos_split,params_num_arvores,params_num_max_atributos,params_produndidade,params_profundidade,state
0,0,NaN,2024-10-27 12:59:18.115507,2024-10-27 12:59:18.326924,0 days 00:00:00.211417,friedman_mse,15,15,23,3.932497,NaN,NaN,FAIL
1,1,NaN,2024-10-27 13:03:40.194593,2024-10-27 13:03:40.368583,0 days 00:00:00.173990,squared_error,9,2,9,4.775030,NaN,NaN,FAIL
2,2,12.377664,2024-10-27 13:06:21.705643,2024-10-27 13:06:29.225864,0 days 00:00:07.520221,squared_error,1,5,79,0.532732,NaN,NaN,COMPLETE
3,3,12.114541,2024-10-27 13:06:29.307954,2024-10-27 13:06:32.920292,0 days 00:00:03.612338,squared_error,4,19,80,0.312661,NaN,NaN,COMPLETE
4,4,12.356317,2024-10-27 13:06:32.983262,2024-10-27 13:06:39.682987,0 days 00:00:06.699725,squared_error,1,2,84,0.249538,NaN,NaN,COMPLETE
...,...,...,...,...,...,...,...,...,...,...,...,...,...
929,929,12.025878,2024-11-16 16:24:07.331212,2024-11-16 16:26:08.369101,0 days 00:02:01.037889,squared_error,2,29,410,0.662329,NaN,NaN,COMPLETE
930,930,12.034336,2024-11-16 16:26:08.446216,2024-11-16 16:29:08.664242,0 days 00:03:00.218026,squared_error,2,33,432,0.926916,NaN,NaN,COMPLETE
931,931,NaN,2024-11-16 16:58:13.160479,2024-11-16 16:58:31.798754,0 days 00:00:18.638275,squared_error,3,27,421,0.506182,NaN,NaN,FAIL
932,932,11.965496,2024-11-16 17:14:18.727548,2024-11-16 17:15:41.617169,0 days 00:01:22.889621,squared_error,3,27,420,0.061848,NaN,NaN,COMPLETE


In [25]:
from sklearn.model_selection import cross_val_score

NUM_FOLDS = 40
SEMENTE_ALEATORIA = 20404
modelo_ad = cria_instancia_modelo(melhor_trial)

metricas = cross_val_score(
    modelo_ad,    #instância
    X,            #dados de atributos
    y,            #dados de target
    cv=NUM_FOLDS, #número de folds
    scoring="neg_root_mean_squared_error", #quanto maior, melhor pelo scikit
)

print("As métricas foram:", metricas)
print()
print("A média das métricas é de:", np.abs(metricas.mean()))

As métricas foram: [-13.00532405 -15.90673954 -11.26655255 -13.53130411 -13.76392681
 -16.79243571 -13.43994276 -14.47297618 -15.09044338 -15.35012649
 -14.35886976 -17.08862334 -15.21866152 -14.2293897  -14.31394589
 -13.39369785 -15.79146112 -11.91858916 -14.6411019  -13.10999527
 -10.46818888 -13.70407045 -10.77050101 -11.86314094  -9.87605856
  -9.4891744  -10.15926946 -10.12671749  -8.04014584  -8.29744475
  -7.9428839   -8.45585984  -7.02848526  -6.89751277  -9.31405929
  -9.45737504  -9.00331152  -7.94237044  -9.74484099  -8.13217421]

A média das métricas é de: 11.834942302997723


## Otimizando hiperparâmetros com o optuna para o regressor KNN

<p style="text-align:justify;"> Nessa momento, irá realizar as mesmas operações passadas, contudo o estudo será dedicado para o modelo KNN. Se esse apresentar um RMSE menor que o regressor de floresta aleatória, será esse o modelo escolhido para realizar a predição de sobrevida de um paciente predito como não curado pelo <strong>modelo classificador</strong>.



In [55]:
df = pd.read_csv("msk_met_2021_clinical_data (1).tsv", sep = "\t")

df['Curado'] = pd.NA
df ['Sobrevivencia (Meses)'] = pd.NA

for i, meses in enumerate(df['Overall Survival (Months)']):
    
    if meses >= 60:
        df.loc [i, 'Curado'] = 1
        
    else:
        if df.loc [i, 'Overall Survival Status'] == '1:DECEASED':
            df.loc [i, 'Curado'] = 0
            df.loc [i, 'Sobrevivencia (Meses)'] = meses

df = df.reindex(ATRIBUTOS + TARGET, axis=1)
df = df.dropna()
df = df.convert_dtypes()

#### Separando dados de treino e teste

In [45]:
from sklearn.model_selection import train_test_split

TAMANHO_TESTE = 0.1
SEMENTE_ALEATORIA = 61455

indices = df.index
indices_treino, indices_teste = train_test_split(
    indices, test_size=TAMANHO_TESTE, random_state=SEMENTE_ALEATORIA
)

df_treino = df.loc[indices_treino]
df_teste = df.loc[indices_teste]

X_treino = df_treino.reindex(ATRIBUTOS, axis=1).values
y_treino = df_treino.reindex(TARGET, axis=1).values.ravel()

X_teste = df_teste.reindex(ATRIBUTOS, axis=1).values
y_teste = df_teste.reindex(TARGET, axis=1).values.ravel()

X = df.reindex(ATRIBUTOS, axis=1).values
y = df.reindex(TARGET, axis=1).values.ravel()

#### Normalizando os atributos de treino

<p style="text-align:justify;"> Normalizar os dados para serem treinados é uma característica do modelo KNN, então, nesse ponto, ele diferencia de como foi treinado o modelo anterior. Os dados tiveram uma normalização padrão.

In [46]:
from sklearn.preprocessing import StandardScaler
normalizador = StandardScaler()
normalizador.fit(X_treino)
X_norm = normalizador.transform(X_treino)

#### Criando as função de instaciar o modelo

In [47]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import cross_val_score

def cria_instancia_modelo(trial):
    """Cria uma instância do modelo.

    Args:
      trial: objeto tipo Trial do optuna.

    Returns:
      Uma instância do modelo desejado.

    """
    parametros = {
        "n_neighbors": trial.suggest_int("vizinhos", 4, 20),
        "weights": trial.suggest_categorical(
            "pesos", ["uniform", "distance"]
        ),
        "p": trial.suggest_float(
            "parametro_potencia", 2, 10, log=True
        )
    }

    model = KNeighborsRegressor(**parametros)

    return model

#### Criando a função objetivo

In [49]:
from sklearn.model_selection import cross_val_score

def funcao_objetivo(trial, X, y, num_folds):
    """Função objetivo do optuna

    Referencia:
      https://medium.com/@walter_sperat/ using-optuna-with-sklearn-the-right-way-part-1-6b4ad0ab2451

    """
    modelo = cria_instancia_modelo(trial)

    metricas = cross_val_score(
        modelo,
        X,
        y,
        scoring="neg_root_mean_squared_error",
        cv=num_folds,
    )

    return -metricas.mean()

#### Criando o documento que irá salvar o estudo acrecar dos melhores hiperparâmetros para o KNN

In [50]:
from optuna import create_study

NOME_DO_ESTUDO = "KNN_comparativo"

objeto_de_estudo = create_study(
    direction="minimize",
    study_name=NOME_DO_ESTUDO,
    storage=f"sqlite:///{NOME_DO_ESTUDO}.db",
    load_if_exists=True,
)

[I 2024-11-16 17:34:01,200] Using an existing study with name 'KNN_comparativo' instead of creating a new one.


#### Função objetivo parcial

In [51]:
NUM_FOLDS = 50

def funcao_objetivo_parcial(trial):
    return funcao_objetivo(trial, X_treino, y_treino, NUM_FOLDS)

#### Otimizando com o optuna

In [52]:
NUM_TENTATIVAS = 200

objeto_de_estudo.optimize(funcao_objetivo_parcial, n_trials=NUM_TENTATIVAS)

[I 2024-11-16 17:34:22,339] Trial 453 finished with value: 12.028049218643599 and parameters: {'vizinhos': 20, 'pesos': 'uniform', 'parametro_potencia': 6.5269236364509045}. Best is trial 453 with value: 12.028049218643599.
[I 2024-11-16 17:34:24,155] Trial 454 finished with value: 12.01894168954165 and parameters: {'vizinhos': 20, 'pesos': 'uniform', 'parametro_potencia': 4.718217739207915}. Best is trial 454 with value: 12.01894168954165.
[I 2024-11-16 17:34:25,802] Trial 455 finished with value: 12.024990130205563 and parameters: {'vizinhos': 20, 'pesos': 'uniform', 'parametro_potencia': 5.888969157264352}. Best is trial 454 with value: 12.01894168954165.
[I 2024-11-16 17:34:27,756] Trial 456 finished with value: 12.026849064702798 and parameters: {'vizinhos': 20, 'pesos': 'uniform', 'parametro_potencia': 8.509029170702934}. Best is trial 454 with value: 12.01894168954165.
[I 2024-11-16 17:34:29,677] Trial 457 finished with value: 12.247202806579041 and parameters: {'vizinhos': 11, 

In [53]:
melhor_trial = objeto_de_estudo.best_trial

print(f"Número do melhor trial: {melhor_trial.number}")
print(f"Parâmetros do melhor trial: {melhor_trial.params}")

Número do melhor trial: 454
Parâmetros do melhor trial: {'vizinhos': 20, 'pesos': 'uniform', 'parametro_potencia': 4.718217739207915}


### Estimando a performance do modelo KNN

In [54]:
from sklearn.model_selection import cross_val_score

NUM_FOLDS = 40
SEMENTE_ALEATORIA = 20404
modelo_ad = cria_instancia_modelo(melhor_trial)

metricas = cross_val_score(
    modelo_ad,    #instância
    X,            #dados de atributos
    y,            #dados de target
    cv=NUM_FOLDS, #número de folds
    scoring="neg_root_mean_squared_error", #quanto maior, melhor pelo scikit
)

print("As métricas foram:", metricas)
print()
print("A média das métricas é de:", np.abs(metricas.mean()))

As métricas foram: [-12.96641637 -14.28304232 -12.98832693 -13.77681775 -13.27256135
 -13.74230308 -14.76950157 -14.68349644 -15.53347737 -15.66091959
 -16.18583361 -16.47896872 -14.35992184 -15.12470355 -13.64962102
 -14.29325765 -14.15597946 -13.66791884 -13.43344439 -12.5203145
 -12.25026824 -11.40833176 -11.1874909  -10.90033856 -11.04956207
 -10.64862012  -9.63099871  -9.96271923  -8.84218549  -9.16705102
  -8.77614686  -8.34914463  -8.28904698  -8.01161565  -8.39719576
  -8.42896354  -8.55851447  -8.17734196  -8.35568679  -7.53583963]

A média das métricas é de: 11.836847217926097


### Conclusão

<p style="text-align:justify;"> Ao encontrar o RMSE do baseline, temos um ponto de partida, ou seja, sabe-se que qualquer modelo que apresente um RMSE, a métrica de performance que escolhemos utilizar,  maior que esse valor não nos gera um bom modelo. A partir desse ponto, começou-se o estudo para selecionar atributos, o que utilizamos a técnica de seleção de atributos pelo fator de inflação de variância (VIF), assim como escolher o melhor conjunto de hiperparâmetros pelo optuna. Ademais, a seleção de melhores atributos foi realizada de acordo com os dois modelos avaliados: floresta aleatória e KNN. Com a métrica de performande de cada um desses modelos, pode-se observar o esperado: os resultados são muito próximos, até próximo do próprio baseline, isso significa que os dados que estamos lidando são bastante difíceis de serem preditos e que há bastante estudo para ser realizado ainda, desde acrescentar mais atributos acerca do paciente, até encontrar novos modelos que consigam lidar com a variabilidade muito sucinta sobre saúde de um paciente. Contudo, precisou-se escolher um modelo e o que ainda apresentou o menor RMSE foi o regressor de floresta aleatória.

### Referências

<p style="text-align:justify;">[1] Nguyen, Bastien, Christopher Fong, Anisha Luthra, Shaleigh A. Smith, Renzo G. DiNatale, Subhiksha Nandakumar, Henry Walch, et al. “Genomic Characterization of Metastatic Patterns from Prospective Clinical Sequencing of 25,000 Patients”. Cell 185, nº 3 (3 de fevereiro de 2022): 563-575.e11. https://doi.org/10.1016/j.cell.2022.01.003.
<p style="text-align:justify;">[2] scikit-learn. “6.3. Preprocessing Data”. Acesso em 9 de outubro de 2024. https://scikit-learn/stable/modules/preprocessing.html.
<p style="text-align:justify;">[3] “ATP-203 1.1 - Tratamento de dados”. Acesso em 9 de outubro de 2024. 
<p style="text-align:justify;">[4] "ATP-203 2.1 - Aprendizado de máquina, k-NN e métricas". Acesso em 16 de novembro de 2024.
<p style="text-align:justify;">[5] “ATP-203 5.1 - Floresta aleatória”. Acesso em 9 de outubro de 2024. 
<p style="text-align:justify;">[6] "ATP-203 6.1 - Otimização de hiperparâmetros com optuna". Acesso em 16 de novembro de 2024.
<p style="text-align:justify;">[7] "ATP-203 7.1 - Seleção de atributos". Acesso em 16 de novembro de 2024.
<p style="text-align:justify;">[8] "ATP-203 9.1 - Classificação binária". Acesso em 10 de novembro de 2024. 
